In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install numpy pandas wfdb scikit-learn torch torchvision torchaudio


In [ ]:
# import os
# import numpy as np
# import pandas as pd
# import wfdb
# import ast
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import classification_report

# # Configuration
# class Config:
#     ECG_FOLDER = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/"
#     METADATA_FILE = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv"
#     SCP_FILE = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/scp_statements.csv"
#     SIGNAL_LENGTH = 1000
#     NUM_LEADS = 12
#     BATCH_SIZE = 32
#     EPOCHS = 55
#     LR = 1e-3
#     HIDDEN_SIZE = 128
#     NUM_LAYERS = 2
#     DROPOUT = 0.3
#     TEST_SIZE = 0.2
#     VAL_SIZE = 0.1
#     RANDOM_STATE = 42

# # Dataset
# class ECGDataset(Dataset):
#     def __init__(self, df, config):
#         self.df = df.reset_index(drop=True)
#         self.config = config

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         path = os.path.join(self.config.ECG_FOLDER, row['filename_lr'])
#         sig, _ = wfdb.rdsamp(path)
#         sig = sig.T
#         # Normalize
#         sig = (sig - sig.mean(axis=1, keepdims=True)) / (sig.std(axis=1, keepdims=True) + 1e-8)
#         # Pad/Trim
#         if sig.shape[1] < self.config.SIGNAL_LENGTH:
#             pad = self.config.SIGNAL_LENGTH - sig.shape[1]
#             sig = np.pad(sig, ((0,0),(0,pad)))
#         else:
#             sig = sig[:, :self.config.SIGNAL_LENGTH]
#         return torch.tensor(sig.T, dtype=torch.float32), torch.tensor(row['class_id'], dtype=torch.long)

# # LSTM Model
# class LSTMClassifier(nn.Module):
#     def __init__(self, config, num_classes):
#         super().__init__()
#         self.lstm = nn.LSTM(
#             input_size=config.NUM_LEADS,
#             hidden_size=config.HIDDEN_SIZE,
#             num_layers=config.NUM_LAYERS,
#             batch_first=True,
#             dropout=config.DROPOUT,
#             bidirectional=True
#         )
#         self.classifier = nn.Sequential(
#             nn.Linear(config.HIDDEN_SIZE * 2, config.HIDDEN_SIZE),
#             nn.ReLU(),
#             nn.Dropout(config.DROPOUT),
#             nn.Linear(config.HIDDEN_SIZE, num_classes)
#         )

#     def forward(self, x):
#         out, _ = self.lstm(x)
#         out = out[:, -1, :]  # Last timestep
#         return self.classifier(out)

# # Preprocessing
# def preprocess(config):
#     df = pd.read_csv(config.METADATA_FILE)
#     scp_df = pd.read_csv(config.SCP_FILE, index_col=0)

#     def get_superclass(code_str):
#         d = ast.literal_eval(code_str)
#         filtered = {k: v for k, v in d.items() if k in scp_df.index and scp_df.loc[k, 'diagnostic'] == 1}
#         if not filtered: return 'NORM'
#         best = max(filtered, key=filtered.get)
#         return scp_df.loc[best, 'diagnostic_class'] or 'NORM'

#     df['diagnostic_superclass'] = df['scp_codes'].apply(get_superclass)
#     le = LabelEncoder()
#     df['class_id'] = le.fit_transform(df['diagnostic_superclass'])

#     train_idx, test_idx = train_test_split(df.index, test_size=config.TEST_SIZE,
#                                            stratify=df['class_id'], random_state=config.RANDOM_STATE)
#     train_meta = df.loc[train_idx].reset_index(drop=True)
#     test_meta = df.loc[test_idx].reset_index(drop=True)

#     train_idx, val_idx = train_test_split(train_meta.index, test_size=config.VAL_SIZE,
#                                           stratify=train_meta['class_id'], random_state=config.RANDOM_STATE)

#     return train_meta.loc[train_idx], train_meta.loc[val_idx], test_meta, le

# # Training + Evaluation
# def train_model():
#     config = Config()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     train_df, val_df, test_df, label_encoder = preprocess(config)
#     num_classes = len(label_encoder.classes_)

#     model = LSTMClassifier(config, num_classes).to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=config.LR)
#     criterion = nn.CrossEntropyLoss()

#     train_loader = DataLoader(ECGDataset(train_df, config), batch_size=config.BATCH_SIZE, shuffle=True)
#     val_loader = DataLoader(ECGDataset(val_df, config), batch_size=config.BATCH_SIZE)
#     test_loader = DataLoader(ECGDataset(test_df, config), batch_size=config.BATCH_SIZE)

#     print(f"Training on {device}")
#     for epoch in range(1, config.EPOCHS + 1):
#         model.train()
#         total_loss, total_correct, total_samples = 0, 0, 0
#         for x, y in train_loader:
#             x, y = x.to(device), y.to(device)
#             optimizer.zero_grad()
#             preds = model(x)
#             loss = criterion(preds, y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item() * y.size(0)
#             total_correct += (preds.argmax(1) == y).sum().item()
#             total_samples += y.size(0)
#         train_loss = total_loss / total_samples
#         train_acc = total_correct / total_samples

#         model.eval()
#         with torch.no_grad():
#             val_loss, val_correct, val_total = 0, 0, 0
#             for x, y in val_loader:
#                 x, y = x.to(device), y.to(device)
#                 preds = model(x)
#                 loss = criterion(preds, y)
#                 val_loss += loss.item() * y.size(0)
#                 val_correct += (preds.argmax(1) == y).sum().item()
#                 val_total += y.size(0)
#         val_loss /= val_total
#         val_acc = val_correct / val_total
#         print(f"Epoch {epoch:2d} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

#     # Final test evaluation
#     model.eval()
#     all_preds, all_labels = [], []
#     with torch.no_grad():
#         for x, y in test_loader:
#             x = x.to(device)
#             out = model(x)
#             preds = out.argmax(1).cpu().numpy()
#             all_preds.extend(preds)
#             all_labels.extend(y.numpy())
#     print("\nFinal Classification Report:")
#     print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

# if __name__ == "__main__":
#     train_model()


In [ ]:
import os
import numpy as np
import pandas as pd
import wfdb
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Configuration
class Config:
    ECG_FOLDER = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/"
    METADATA_FILE = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/ptbxl_database.csv"
    SCP_FILE = "/kaggle/input/ptb-xl-dataset/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/scp_statements.csv"
    SIGNAL_LENGTH = 1000
    NUM_LEADS = 12
    BATCH_SIZE = 32
    EPOCHS = 55
    LR = 1e-3
    # LSTM specific
    HIDDEN_SIZE = 128
    NUM_LAYERS = 2
    # CNN specific
    CNN_OUPUT_CHANNELS = 128 # Should be the input to the LSTM
    # Shared
    DROPOUT = 0.3
    # Data splitting
    TEST_SIZE = 0.2
    VAL_SIZE = 0.1
    RANDOM_STATE = 42

# Dataset class remains unchanged
class ECGDataset(Dataset):
    def __init__(self, df, config):
        self.df = df.reset_index(drop=True)
        self.config = config

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.config.ECG_FOLDER, row['filename_lr'])
        # Use wfdb.rdsamp to read both the signal and metadata
        sig, _ = wfdb.rdsamp(path)
        sig = sig.T # Transpose to get (12, 5000)
        # Normalize each lead independently
        sig = (sig - sig.mean(axis=1, keepdims=True)) / (sig.std(axis=1, keepdims=True) + 1e-8)
        # Pad or trim the signal to the desired length
        if sig.shape[1] < self.config.SIGNAL_LENGTH:
            pad_width = self.config.SIGNAL_LENGTH - sig.shape[1]
            sig = np.pad(sig, ((0,0),(0,pad_width)))
        else:
            sig = sig[:, :self.config.SIGNAL_LENGTH]
        # Transpose back for LSTM: (SIGNAL_LENGTH, NUM_LEADS)
        return torch.tensor(sig.T, dtype=torch.float32), torch.tensor(row['class_id'], dtype=torch.long)

# NEW: Hybrid CNN-LSTM Model
class CNN_LSTM_Classifier(nn.Module):
    def __init__(self, config, num_classes):
        super().__init__()
        # 1D CNN Feature Extractor
        self.cnn = nn.Sequential(
            # Conv Block 1
            nn.Conv1d(in_channels=config.NUM_LEADS, out_channels=64, kernel_size=7, stride=1, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # Conv Block 2
            nn.Conv1d(in_channels=64, out_channels=config.CNN_OUPUT_CHANNELS, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )

        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=config.CNN_OUPUT_CHANNELS, # Input features are the output channels of the CNN
            hidden_size=config.HIDDEN_SIZE,
            num_layers=config.NUM_LAYERS,
            batch_first=True,
            dropout=config.DROPOUT,
            bidirectional=True
        )

        # Classifier Head
        self.classifier = nn.Sequential(
            nn.Linear(config.HIDDEN_SIZE * 2, config.HIDDEN_SIZE), # *2 for bidirectional
            nn.ReLU(),
            nn.Dropout(config.DROPOUT),
            nn.Linear(config.HIDDEN_SIZE, num_classes)
        )

    def forward(self, x):
        # Input shape for CNN: (batch_size, num_leads, signal_length)
        # Current input x shape: (batch_size, signal_length, num_leads)
        # Permute to match Conv1d expectation
        x = x.permute(0, 2, 1)

        # Pass through CNN
        x = self.cnn(x) # Shape: (batch_size, cnn_output_channels, new_signal_length)

        # Input shape for LSTM: (batch_size, sequence_length, features)
        # Permute again to match LSTM expectation
        x = x.permute(0, 2, 1)

        # Pass through LSTM
        out, _ = self.lstm(x)

        # Get the output of the last time step
        out = out[:, -1, :]

        # Pass through the classifier
        return self.classifier(out)

# Preprocessing function remains unchanged
def preprocess(config):
    df = pd.read_csv(config.METADATA_FILE)
    scp_df = pd.read_csv(config.SCP_FILE, index_col=0)

    # Convert scp_codes from string to dictionary
    df['scp_codes'] = df['scp_codes'].apply(ast.literal_eval)

    def get_diagnostic_superclass(scp_codes):
        diagnostic_classes = []
        for code, confidence in scp_codes.items():
            if code in scp_df.index and scp_df.loc[code, 'diagnostic'] == 1:
                superclass = scp_df.loc[code, 'diagnostic_class']
                if not pd.isna(superclass):
                    diagnostic_classes.append(superclass)
        
        if not diagnostic_classes:
            return 'NORM' # Default to NORM if no diagnostic class is found
        
        # Return the most frequent diagnostic class if multiple are present
        return max(set(diagnostic_classes), key=diagnostic_classes.count)

    df['diagnostic_superclass'] = df['scp_codes'].apply(get_diagnostic_superclass)
    
    le = LabelEncoder()
    df['class_id'] = le.fit_transform(df['diagnostic_superclass'])

    # Splitting data
    train_val_df, test_df = train_test_split(df, test_size=config.TEST_SIZE,
                                             stratify=df['class_id'], random_state=config.RANDOM_STATE)
    train_df, val_df = train_test_split(train_val_df, test_size=config.VAL_SIZE / (1 - config.TEST_SIZE),
                                        stratify=train_val_df['class_id'], random_state=config.RANDOM_STATE)

    return train_df, val_df, test_df, le


# Training + Evaluation function updated to use the new model
def train_model():
    config = Config()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_df, val_df, test_df, label_encoder = preprocess(config)
    num_classes = len(label_encoder.classes_)

    # Use the new CNN_LSTM_Classifier
    model = CNN_LSTM_Classifier(config, num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LR)
    criterion = nn.CrossEntropyLoss()

    # DataLoaders remain the same
    train_loader = DataLoader(ECGDataset(train_df, config), batch_size=config.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(ECGDataset(val_df, config), batch_size=config.BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(ECGDataset(test_df, config), batch_size=config.BATCH_SIZE, shuffle=False)

    print(f"Training on {device} using CNN-LSTM model...")
    for epoch in range(1, config.EPOCHS + 1):
        model.train()
        total_loss, total_correct, total_samples = 0, 0, 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            preds = model(x)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * y.size(0)
            total_correct += (preds.argmax(1) == y).sum().item()
            total_samples += y.size(0)
        train_loss = total_loss / total_samples
        train_acc = total_correct / total_samples

        model.eval()
        with torch.no_grad():
            val_loss, val_correct, val_total = 0, 0, 0
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x)
                loss = criterion(preds, y)
                val_loss += loss.item() * y.size(0)
                val_correct += (preds.argmax(1) == y).sum().item()
                val_total += y.size(0)
        val_loss /= val_total
        val_acc = val_correct / val_total
        print(f"Epoch {epoch:2d} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    # Final test evaluation
    print("\n--- Final Test Evaluation ---")
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            out = model(x)
            preds = out.argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(y.numpy())
            
    print("\nFinal Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

if __name__ == "__main__":
    train_model()